In [20]:
import numpy as np
import pandas as pd
import geopandas as gpd

import statsmodels.stats.api as sm
from pysal.lib import weights
from pysal.model import spreg

from matplotlib import pyplot as plt
import seaborn as sns
import contextily as cx

In [5]:
df = pd.read_csv("realtor_data.csv")
df

,address/street,baths,beds,coordinates/latitude,coordinates/longitude,garage,id,listPrice,is_manual,name,sqft,status,stories,type,units,year_built
0,809 S Aurora St,2.0,5.0,42.430851,-76.496069,0,4208852367,2625.0,False,NaN,1300.0,for_rent,1,single_family,1,NaN
1,419 Triphammer Rd,2.0,6.0,42.460409,-76.481516,4,4733433667,5000.0,False,NaN,3200.0,for_rent,2,single_family,1,1956.0
2,517 S Aurora St,2.0,6.0,42.435014,-76.494344,0,4090421202,4350.0,False,NaN,2457.0,for_rent,1,single_family,1,NaN
3,3 Hudson Pl,1.0,2.0,42.430688,-76.493156,0,4199200488,1600.0,False,NaN,900.0,for_rent,1,single_family,1,NaN
4,600 Warren Rd Apt 4-2,1.5,1.5,42.477345,-76.467567,0,9934501911,1881.0,True,Warren Wood,894.0,for_rent,1,apartment,181,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,519 E Buffalo St,1.0,2.0,42.441285,-76.490701,0,3834217847,1200.0,False,"Moll Properties, Inc.",0.0,for_rent,1,other,7,NaN
229,106 Wiedmaier Ct,2.0,3.0,42.413681,-76.449093,0,3610937531,3244.0,True,Ithacor,1476.0,for_rent,1,apartment,2,2007.0
230,409 College Ave,1.0,1.5,42.442255,-76.484963,0,9523596613,4095.0,True,"Student Agencies Properties, Inc.",725.0,for_rent,1,apartment,64,NaN
231,103 Wiedmaier Ct,1.5,3.0,42.413895,-76.448010,0,3609558692,1450.0,True,Ithacor,1144.5,for_rent,1,apartment,2,2015.0


In [17]:
variable_names = ['baths', 'beds', 'garage', 'sqft']

In [8]:
df = df.dropna(subset=['coordinates/latitude', 'coordinates/longitude'])

In [12]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['coordinates/latitude'], df['coordinates/longitude']))
gdf.explore(tiles='CartoDB dark_matter')

In [22]:
w_kernel = weights.distance.Kernel.from_dataframe(gdf, k=5)

In [23]:
lag_model = spreg.GM_Lag(df[["listPrice"]].values, df[variable_names].values,
                         w=w_kernel,
                         name_y="log_price",name_x=variable_names,
)

Exception: All entries on diagonal must equal 0.